In [1]:
import models
from permutation_metrics import rank_similarities
import utils
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np
import matplotlib.pyplot as plt
from torch import optim
from torch.autograd import Variable


%load_ext autoreload
%autoreload 2

torch.manual_seed(42)

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

Y = torch.tensor(np.load('../datasets/floored_exp_uncorrel.npy'), dtype=dtype)
Yt = Y.transpose(0,1)
real_data = torch.tensor(np.genfromtxt('../datasets/real_data.csv', delimiter = ','))
A_true = np.load('../datasets/students_uncorrel.npy')
D_true = np.load('../datasets/questions_uncorrel.npy')

# We assume we know the relevant concept of each question beforehand
concepts = np.nonzero(D_true)
num_students, num_concepts = A_true.shape
num_questions = D_true.shape[0]
guess_prob = 1/5

In [2]:
def accuracy(R_pred, R_true):
    R_pred = R_pred.data.numpy()
    R_true = R_true.data.numpy()
    R_pred_cpy = np.copy(R_pred)
    R_pred_cpy[R_pred_cpy > 0.5] = 1
    R_pred_cpy[R_pred_cpy <= 0.5] = 0
    print("Accuracy: {}".format(np.sum(R_pred_cpy == R_true) / (R_true.shape[0]*R_true.shape[1])))
    return np.sum(R_pred_cpy == R_true) / (R_true.shape[0]*R_true.shape[1])

In [3]:
def rmse_min_max(A,B):
    a_norm = (A - A.min())/(A.max()-A.min())
    b_norm = (B - B.min())/(B.max()-B.min())
    return np.sqrt(np.mean(np.square(a_norm-b_norm))) 

# Run on Simulated Data

In [4]:
#Preparing to train simulated data set
observations = Yt
idx_train = int(0.7*observations.size()[1])
idx_val = int(0.8*observations.size()[1])
train = observations[:,:idx_train]
val = observations[:,idx_train:idx_val]
test = observations[:,idx_val:]
idx_test = observations.size()[1] - idx_val
idx_val = idx_val-idx_train

## Simple RNN to predict students performance

In [8]:
n_epochs = 1000
hidden_size = 128
layers = 8
batch_size = 100 #needs to have idx_train, idx_val, and idx_train as a multiple
rate = 0.002
dropout = 0.2

model = models.RNN_Model(hidden_size, batch_size, layers, dropout)
criterion = nn.functional.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [ ]:
#Training
losses = np.zeros(n_epochs) # For plotting
num_batches = int(idx_train/batch_size)
preds = torch.zeros(train.size()[0]-1,train.size()[1])

for epoch in range(n_epochs):
    for batch in range(num_batches):
        inputs = Variable(train[:-1,batch*batch_size:(batch+1)*batch_size])
        targets = Variable(train[1:,batch*batch_size:(batch+1)*batch_size])

        outputs, hidden = model(inputs, None)
        preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
        
        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        #if batch % 2 == 0:
            #print("Batch Done")
    if epoch % 20 == 0:
        losses[epoch] += loss.data[0]
        print(epoch, loss.data[0])
        accuracy(preds, train[1:,:])

/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(0.6946)
Accuracy: 0.49226551226551224
20 tensor(0.6911)
Accuracy: 0.5384992784992785
40 tensor(0.6881)
Accuracy: 0.5402741702741702
60 tensor(0.6872)
Accuracy: 0.5427705627705628
80 tensor(0.6682)
Accuracy: 0.5722510822510822
100 tensor(0.6736)
Accuracy: 0.5538239538239538
120 tensor(0.6683)
Accuracy: 0.5658441558441558
140 tensor(0.6608)
Accuracy: 0.5811832611832611
160 tensor(0.6304)
Accuracy: 0.6290909090909091
180 tensor(0.5881)
Accuracy: 0.672958152958153
200 tensor(0.6575)
Accuracy: 0.5974747474747475
220 tensor(0.6182)
Accuracy: 0.6441702741702742
240 tensor(0.6057)
Accuracy: 0.6473593073593074
260 tensor(0.5746)
Accuracy: 0.6821356421356422
280 tensor(0.6102)
Accuracy: 0.6397113997113997
300 tensor(0.5990)
Accuracy: 0.6448484848484849
320 tensor(0.5712)
Accuracy: 0.6805194805194805
340 tensor(0.5631)
Accuracy: 0.6911111111111111
360 tensor(0.5479)
Accuracy: 0.7032178932178932
380 tensor(0.5430)
Accuracy: 0.7071139971139971
400 tensor(0.5399)
Accuracy: 0.708427128427128

In [58]:
#Validation
num_batches = int(idx_val/batch_size)
preds = torch.zeros(val.size()[0]-1,val.size()[1])

for batch in range(num_batches):
    inputs = Variable(val[:-1,batch*batch_size:(batch+1)*batch_size])
    targets = Variable(val[1:,batch*batch_size:(batch+1)*batch_size])

    outputs, hidden = model(inputs, None)
    preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
    
accuracy(preds, val[1:,:])

Accuracy: 0.526969696969697


0.526969696969697

In [ ]:
#Test
num_batches = int(idx_test/batch_size)
preds = torch.zeros(test.size()[0]-1,test.size()[1])

for batch in range(num_batches):
    inputs = Variable(test[:-1,batch*batch_size:(batch+1)*batch_size])
    targets = Variable(test[1:,batch*batch_size:(batch+1)*batch_size])

    outputs, hidden = model(inputs, None)
    preds[:,batch*batch_size:(batch+1)*batch_size] = outputs

accuracy(preds, test[1:,:])

## Skill RNN to predict students performance

In [6]:
n_epochs = 500
batch_size = 100
average = True
sigmoid = True
#concepts = ([0,1,2,3,4,5,6,7,8],[0,0,0,0,0,0,0,0,0])
hidden_size = 128
dropout = 0.5
num_layers = 4
rate = 0.002

model = models.RNN_Skills_Model(average, concepts, num_concepts, num_questions, hidden_size, batch_size, num_layers, dropout, sigmoid)
criterion = nn.functional.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [7]:
#Training
losses = np.zeros(n_epochs) # For plotting
num_batches = int(idx_train/batch_size)
preds = torch.zeros(train.size()[0]-1,train.size()[1])
A_pred = torch.zeros(train.size()[1],num_concepts)

for epoch in range(n_epochs):

    for batch in range(num_batches):
        inputs = Variable(train[:-1,batch*batch_size:(batch+1)*batch_size])
        targets = Variable(train[1:,batch*batch_size:(batch+1)*batch_size])

        outputs, hidden, skills, D = model(inputs, None)
        preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
        A_pred[batch*batch_size:(batch+1)*batch_size,:] = skills 
        
        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    losses[epoch] += loss.data[0]

    if epoch % 20 == 0:
        print(epoch, loss.data[0])
        accuracy(outputs,targets)
        print('RMSE A: {}'.format(rmse_min_max(A_pred.data.numpy(), A_true[:idx_train])))
        print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))

/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(0.7267)
Accuracy: 0.492020202020202
RMSE A: 0.3078810682157786
RMSE D: 0.34034965162084274
20 tensor(0.6611)
Accuracy: 0.6188888888888889
RMSE A: 0.49769154927109654
RMSE D: 0.3696441336369644
40 tensor(0.6359)
Accuracy: 0.6941414141414142
RMSE A: 0.4687363139392497
RMSE D: 0.40833851940428645
60 tensor(0.6188)
Accuracy: 0.6984848484848485
RMSE A: 0.4480377938732995
RMSE D: 0.4238140847588058
80 tensor(0.6080)
Accuracy: 0.703030303030303
RMSE A: 0.43202380746687274
RMSE D: 0.4266416752777227
100 tensor(0.5999)
Accuracy: 0.7038383838383838
RMSE A: 0.4151657716915532
RMSE D: 0.4265908457015089
120 tensor(0.5937)
Accuracy: 0.7081818181818181
RMSE A: 0.40635122674206164
RMSE D: 0.425449801665857
140 tensor(0.5889)
Accuracy: 0.7078787878787879
RMSE A: 0.3928251106909976
RMSE D: 0.42388758922034303
160 tensor(0.5856)
Accuracy: 0.7088888888888889
RMSE A: 0.383502757163619
RMSE D: 0.42176796542723477
180 tensor(0.5836)
Accuracy: 0.7084848484848485
RMSE A: 0.37826922605255625
RMSE D: 0

In [ ]:
#Validation
num_batches = int(idx_val/batch_size)
preds = torch.zeros(val.size()[0]-1,val.size()[1])
A_pred = torch.zeros(val.size()[1],num_concepts)

for batch in range(num_batches):
    inputs = Variable(val[:-1,batch*batch_size:(batch+1)*batch_size])
    targets = Variable(val[1:,batch*batch_size:(batch+1)*batch_size])

    outputs, hidden, skills, D = model(inputs, None)
    preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
    A_pred[batch*batch_size:(batch+1)*batch_size,:] = skills 
    
print('RMSE A: {}'.format(rmse_min_max(A_pred.data.numpy(), A_true[idx_train:(idx_train+idx_val)])))
print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))
accuracy(preds, val[1:,:])

In [ ]:
#Test
num_batches = int(idx_test/batch_size)
preds = torch.zeros(test.size()[0]-1,test.size()[1])
A_pred = torch.zeros(test.size()[1],num_concepts)

for batch in range(num_batches):
    inputs = Variable(test[:-1,batch*batch_size:(batch+1)*batch_size])
    targets = Variable(test[1:,batch*batch_size:(batch+1)*batch_size])

    outputs, hidden, skills, D = model(inputs, None)
    preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
    A_pred[batch*batch_size:(batch+1)*batch_size,:] = skills 
    
print('RMSE A: {}'.format(rmse_min_max(A_pred.data.numpy(), A_true[(idx_train+idx_val):])))
print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))
accuracy(preds, val[1:,:])

In [14]:
print(rank_similarities(A_true, Y.data.numpy(), (skills.data.numpy() - skills.data.numpy().min())/(skills.data.numpy().max()-skills.data.numpy().min()))['summary'])


        Summary of Ranking Evaluation: 
        The correlations are with the true rankings derived from A_true.
        For the baseline, we get a Kendall Rank correlation of 0.033, with p-value of 0.124, and a Spearman correlation of 0.049, with p-value of 0.125. 
        For the prediction, we get a Kendall Rank correlation of -0.012, with p-value of 0.558, and a Spearman correlation of -0.019, with p-value of 0.546.  
        Which gives us an average difference of -0.0565 versus the baseline. 
        


In [32]:
#Hyperparameter Search
n_epochs = 50
hidden_size = {512,256,128,64}
dropout = np.random.uniform(size=5)
print(dropout)
num_layers = {32,16,8,4}
#l_rate = np.random.uniform(0.01,0.5,5)
l_rate = {0.002,0.01,0.05,0.2,0.5}
print(l_rate)
A_rmse = 1
D_rmse = 1
max_acc = 0
params = {}
concepts = np.nonzero(D_true)
acc = []
h_loss = {}
i=0
for size in hidden_size:
    for drop in dropout:
        for layers in num_layers:
            for rate in l_rate:
                model = models.RNN_Skills_Model(False, concepts, num_concepts, num_questions, size, num_students, layers, drop)
                criterion = nn.functional.binary_cross_entropy
                optimizer = optim.Adam(model.parameters(), lr=rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

                losses = np.zeros(n_epochs)
                
                for epoch in range(n_epochs):

                    inputs = Variable(Yt[:-1])
                    targets = Variable(Yt[1:])

                    outputs, hidden, skills, D = model(inputs, None)

                    optimizer.zero_grad()
                    loss = criterion(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    
                    losses[epoch] += loss.data[0]
                    
                    if epoch>0 and np.abs(losses[epoch]-losses[epoch-1])<0.00005:
                        break

                    if epoch % 10 == 0:
                        print(epoch, loss.data[0])
                        print(accuracy(outputs,targets))
                        print('RMSE A: {}'.format(rmse_min_max(skills.data.numpy(), A_true)))
                        print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))

                
                print(accuracy(outputs,targets))
                if accuracy(outputs,targets)>max_acc:
                    params['accuracy'] = (size,drop,layers,rate)
                    max_acc = accuracy(outputs,targets)
                if rmse_min_max(skills.data.numpy(), A_true)<A_rmse:
                    params['A'] = (size,drop,layers,rate)
                    A_rmse = rmse_min_max(skills.data.numpy(), A_true)
                if rmse_min_max(D.data.numpy(), D_true)<D_rmse:
                    params['D'] = (size,drop,layers,rate)
                    D_rmse = rmse_min_max(D.data.numpy(), D_true)
                acc.append(accuracy(outputs,targets))
                h_loss[i] = losses
                print('done')
                i+=1
                    
print(params)
print(max_acc)
print(A_rmse)
print(D_rmse)
print(acc)

[0.67363352 0.34875009 0.45567222 0.76393018 0.62340218]
{0.5, 0.1, 0.3, 0.2, 0.02}


/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(1.3280)
0.48944444444444446
RMSE A: 0.41321895522657626
RMSE D: 0.46254148619455304
10 tensor(0.8601)
0.4803131313131313
RMSE A: 0.36741722820092293
RMSE D: 0.3854353724191575
0.4787070707070707
done
0 tensor(1.9985)
0.5032929292929293
RMSE A: 0.50919118529673
RMSE D: 0.4492963114053607
10 tensor(0.8606)
0.4785656565656566
RMSE A: 0.36756588297951304
RMSE D: 0.3713725979540209
20 tensor(0.8554)
0.48397979797979795
RMSE A: 0.3641292753668714
RMSE D: 0.42039191691995553
0.48597979797979796
done
0 tensor(1.2593)
0.5110707070707071
RMSE A: 0.48182128081911146
RMSE D: 0.330943770978714
0.4784242424242424
done
0 tensor(2.4409)
0.5184444444444445
RMSE A: 0.47526985063165056
RMSE D: 0.34508595146288723
10 tensor(0.8406)
0.5003636363636363
RMSE A: 0.35819794728253196
RMSE D: 0.43959553735739854
20 tensor(0.8348)
0.5008181818181818
RMSE A: 0.3194916266610254
RMSE D: 0.4446673889964636
0.5054848484848485
done
0 tensor(1.4675)
0.4865959595959596
RMSE A: 0.4976868465537525
RMSE D: 0.387302

KeyboardInterrupt: 